In [1]:
import pandas as pd
from path import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

### Data Preparation

In [2]:
file = Path('crypto_data.csv')

In [3]:
df = pd.read_csv(file, index_col=0)
df.head(10)

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined  \
42      42 Coin    Scrypt       True   PoW/PoS     4.199995e+01   
365     365Coin       X11       True   PoW/PoS              NaN   
404     404Coin    Scrypt       True   PoW/PoS     1.055185e+09   
611   SixEleven   SHA-256       True       PoW              NaN   
808         808   SHA-256       True   PoW/PoS     0.000000e+00   
1337  EliteCoin       X13       True   PoW/PoS     2.927942e+10   
2015  2015 coin       X11       True   PoW/PoS              NaN   
BTC     Bitcoin   SHA-256       True       PoW     1.792718e+07   
ETH    Ethereum    Ethash       True       PoW     1.076842e+08   
LTC    Litecoin    Scrypt       True       PoW     6.303924e+07   

     TotalCoinSupply  
42                42  
365       2300000000  
404        532000000  
611           611000  
808                0  
1337    314159265359  
2015               0  
BTC         21000000  
ETH                0  
LTC         84000000

In [4]:
# List all algorithms in use
algorithms = set(df.Algorithm.values)
print(algorithms)

{'QuBit', 'Progressive-n', 'VBFT', 'SkunkHash v2 Raptor', 'M7 POW', 'T-Inside', 'CryptoNight-lite', 'Argon2d', '536', 'Multiple', 'Equihash1927', 'HybridScryptHash256', 'Exosis', 'CryptoNight Heavy X', 'CryptoNight-V7', 'Ethash', 'Equihash', 'Momentum', 'HMQ1725', 'QUAIT', 'Scrypt', 'NIST5', 'Shabal256', 'PoS', 'Skein', 'Proof-of-Authority', 'Dagger-Hashimoto', 'Argon2', 'BLAKE256', 'Stanford Folding', 'DPoS', 'Quark', 'VeChainThor Authority', 'Groestl', 'Zhash', 'X13', 'Counterparty', 'Proof-of-BibleHash', 'POS 2.0', 'SHA-512', 'SHA-256D', 'C11', 'Lyra2RE', 'CryptoNight-Lite', 'X14', 'Lyra2Z', 'Scrypt-n', 'XEVAN', 'Avesta hash', 'SkunkHash', 'Lyra2REv2', 'Blake2b', 'Ouroboros', 'TRC10', 'Curve25519', 'Rainforest', 'X11', 'Equihash+Scrypt', 'Dagger', 'Semux BFT consensus', 'SHA3-256', '1GB AES Pattern Search', 'ScryptOG', 'NeoScrypt', 'Cryptonight-GPU', 'QuarkTX', 'YescryptR16', 'Wild Keccak', 'SHA-256 + Hive', 'POS 3.0', 'PHI1612', 'Jump Consistent Hash', 'Green Protocol', 'vDPOS', 'C

In [5]:
# print the counts of stocks being currently traded vs. not being currently traded 
df['IsTrading'].value_counts()

True     1144
False     108
Name: IsTrading, dtype: int64

In [6]:
# Select only cryptocurrencies that are being traded
df1 = df.loc[df["IsTrading"]==True]
df1["IsTrading"].value_counts()

True    1144
Name: IsTrading, dtype: int64

In [7]:
# Delete the IsTrading column; it is no longer necessary
df1 = df1.drop("IsTrading", axis=1)
df1.head()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
365    365Coin       X11   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       PoW              NaN          611000
808        808   SHA-256   PoW/PoS     0.000000e+00               0

In [8]:
# Delete all rows with null values
df1 = df1.dropna(axis=0, how='any')
df1.shape

(685, 5)

In [9]:
# Sort values in TotalCoinsMined to identify negative or zero values
df1.TotalCoinsMined.sort_values()

FIII   -5.917978e+09
LBTC    0.000000e+00
RIPO    0.000000e+00
BASH    0.000000e+00
CSH     0.000000e+00
            ...     
QWC     9.955311e+10
NYC     1.430067e+11
GCN     1.630551e+11
BCN     1.840668e+11
BTT     9.899887e+11
Name: TotalCoinsMined, Length: 685, dtype: float64

In [10]:
# Select only rows with positive TotalCoinsMined values
df2 = df1[df1['TotalCoinsMined']>0]
len(df2)

532

In [11]:
# Delete the CoinName column from the original dataframe
df2 = df2.drop('CoinName',axis=1)
df2.head()

Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42      Scrypt   PoW/PoS     4.199995e+01              42
404     Scrypt   PoW/PoS     1.055185e+09       532000000
1337       X13   PoW/PoS     2.927942e+10    314159265359
BTC    SHA-256       PoW     1.792718e+07        21000000
ETH     Ethash       PoW     1.076842e+08               0

In [12]:
# Create dummy variables for columns with string values
x = pd.get_dummies(data=df2, columns = ['Algorithm', 'ProofType'])
print(x.shape)
x.head()

(532, 98)


TotalCoinsMined TotalCoinSupply  Algorithm_1GB AES Pattern Search  \
42       4.199995e+01              42                                 0   
404      1.055185e+09       532000000                                 0   
1337     2.927942e+10    314159265359                                 0   
BTC      1.792718e+07        21000000                                 0   
ETH      1.076842e+08               0                                 0   

      Algorithm_536  Algorithm_Argon2d  Algorithm_BLAKE256  Algorithm_Blake  \
42                0                  0                   0                0   
404               0                  0                   0                0   
1337              0                  0                   0                0   
BTC               0                  0                   0                0   
ETH               0                  0                   0                0   

      Algorithm_Blake2S  Algorithm_Blake2b  Algorithm_C11  ...  \
42                    0                  0              0  ...   
404                   0                  0              0  ...   
1337                  0                  0              0  ...   
BTC                   0                  0              0  ...   
ETH                   0                  0              0  ...   

      ProofType_PoW/PoS  ProofType_PoW/PoS   ProofType_PoW/PoW  \
42                    1                   0                  0   
404                   1                   0                  0   
1337                  1                   0                  0   
BTC                   0                   0                  0   
ETH                   0                   0                  0   

      ProofType_PoW/nPoS  ProofType_Pos  ProofType_Proof of Authority  \
42                     0              0                             0   
404                    0              0                             0   
1337                   0              0                             0   
BTC                    0              0                             0   
ETH                    0              0                             0   

      ProofType_Proof of Trust  ProofType_TPoS  \
42                           0               0   
404                          0               0   
1337                         0               0   
BTC                          0               0   
ETH                          0               0   

      ProofType_Zero-Knowledge Proof  ProofType_dPoW/PoW  
42                                 0                   0  
404                                0                   0  
1337                               0                   0  
BTC                                0                   0  
ETH                                0                   0  

[5 rows x 98 columns]

In [13]:
# Standardize the data
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [14]:
# print the scaled data
x_scaled[0]

array([-0.11710817, -0.1528703 , -0.0433963 , -0.0433963 , -0.0433963 ,
       -0.06142951, -0.07530656, -0.0433963 , -0.06142951, -0.06142951,
       -0.0433963 , -0.0433963 , -0.19245009, -0.06142951, -0.09740465,
       -0.0433963 , -0.11547005, -0.07530656, -0.0433963 , -0.0433963 ,
       -0.15191091, -0.0433963 , -0.13118084, -0.0433963 , -0.0433963 ,
       -0.08703883, -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
       -0.06142951, -0.0433963 , -0.08703883, -0.08703883, -0.08703883,
       -0.0433963 , -0.13118084, -0.13840913, -0.13840913, -0.0433963 ,
       -0.06142951, -0.0433963 , -0.07530656, -0.18168574, -0.0433963 ,
       -0.0433963 , -0.0433963 , -0.07530656, -0.15826614, -0.31491833,
       -0.0433963 , -0.08703883, -0.07530656, -0.06142951,  1.38675049,
       -0.0433963 , -0.0433963 , -0.06142951, -0.0433963 , -0.0433963 ,
       -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
       -0.0433963 , -0.39879994, -0.0433963 , -0.18168574, -0.04

In [15]:
# Identify the numbers of rows and columns in the scaled data
x_scaled.shape

(532, 98)

### Reduce dataset dimensions with PCA

In [16]:
#with 99% of data 
pca = PCA(n_components=0.99)
prin_components = pca.fit_transform(x_scaled)
prin_components.shape

(532, 86)

In [17]:
pca.explained_variance_.sum()

97.35133315075761

In [18]:
# Reduce dimensions with PCA (90%)
pca = PCA(n_components=0.90)
prin_components = pca.fit_transform(x_scaled)
prin_components.shape

(532, 74)

In [19]:
# The sum of the explained variance of the principal components
pca.explained_variance_.sum()

88.47283457566151

In [20]:
# Reduce dimensions with t-SNE
tsne1 = TSNE(perplexity=50)
tsne1Features = tsne1.fit_transform(prin_components)
tsne1Features.shape

(532, 2)

In [21]:
# Plot t-SNE output
x_val = tsne1Features[:,0]
y_val = tsne1Features[:,1]

plt.scatter(x_val,y_val)
plt.show()

In [23]:
# Identify clusters with k-means
inertia = []
k = [1,2,3,4,5,6,7,8,9,10]
for value in k: 
    kmeans = KMeans(n_clusters=value, random_state=0)
    kmeans.fit(prin_components)
    inertia.append(kmeans.inertia_)


C:\Users\Hannah\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [25]:
# Create an elbow plot
elbowPlt = pd.DataFrame({"K":k, "Inertia":inertia})
elbowPlt.plot.line(x="K",y="Inertia")
plt.ylabel("Inertia")
plt.title("Elbow Plot")
plt.xlabel("k")
plt.show

<function matplotlib.pyplot.show(close=None, block=None)>

# Conclusion 

based on the elbow plot and t-SNE analysis, there isn't great evidence to say that the data set has a cluster. 